<a href="https://www.kaggle.com/code/ubaydulloasatullaev/bank-churn-binary-pgs41-autogluon?scriptVersionId=161129356" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Prepare everything

In [1]:
!pip install -U autogluon > /dev/null


from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np


TRAIN = '/kaggle/input/playground-series-s4e1/train.csv'
TEST = '/kaggle/input/playground-series-s4e1/test.csv'
ORIGINAL = '/kaggle/input/bank-customer-churn-prediction/Churn_Modelling.csv'


df_train = pd.read_csv(TRAIN)
df_orig = pd.read_csv(ORIGINAL)
df_test = pd.read_csv(TEST)


# Include the original dataset
df_train = pd.concat((df_train, df_orig), axis=0).drop(['id', 'RowNumber'], axis=1).drop_duplicates().dropna()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.11.0 requires botocore<1.34.23,>=1.33.2, but you have botocore 1.29.165 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
conda 23.11.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
jupyter-events 0.9.0 requires js

# Apply different preprocessing schemes

In [2]:
# Below is simple feature processing from https://www.kaggle.com/code/lordpatil/automl-autogluon


def processing1(df):
    df['Surname_len'] = df['Surname'].apply(len)
    df['Geography'] = df['Geography'].map({'France': 0, 'Spain': 1, 'Germany': 2})
    df['Gender'] = (df['Gender']=='Male').astype(np.int64)
    return df

def processing2(df):
    cols = ['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','EstimatedSalary', 'Exited']
    df = df.loc[df.groupby(cols)['Tenure'].idxmax()].drop_duplicates()
    df = df[(df.NumOfProducts < 4) | (df.HasCrCard == 1)]
    return df

def processing3(df):
    df['IsSenior'] = df['Age'].apply(lambda x: 1 if x >= 60 else 0)
    df['IsActive_by_CreditCard'] = df['HasCrCard'] * df['IsActiveMember']
    df['Products_Per_Tenure'] =  df['Tenure'] / df['NumOfProducts']
    df['AgeCat'] = np.round(df.Age/20).astype('int').astype('category')
    df['Sur_Geo_Gend_Sal'] = df['Surname']+df['Geography']+df['Gender']+np.round(df.EstimatedSalary).astype('str')
    return df


#df_train = processing1(df_train)

df_train = processing2(df_train)

df_train = processing3(df_train)
#df_test = processing1(df_test)
df_test = processing3(df_test)

# Fit the AutoGluon predictor

In [3]:
train = TabularDataset(df_train)
test = TabularDataset(df_test)


automl = TabularPredictor(label='Exited', problem_type='binary', eval_metric='roc_auc')
automl.fit(train, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240131_064910"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240131_064910/ds_sub_fit/sub_fit_ho.
2024-01-31 06:49:12,070	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning Au

# Explore the leaderboard

In [4]:
automl.leaderboard()


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.897283,roc_auc,122.673525,2267.090975,0.033096,60.307057,3,True,18
1,CatBoost_BAG_L2,0.896596,roc_auc,102.386087,1844.000767,2.114902,163.827751,2,True,17
2,WeightedEnsemble_L2,0.895909,roc_auc,62.095538,1470.974447,0.033869,41.606848,2,True,12
3,CatBoost_BAG_L1,0.895646,roc_auc,2.997290,1003.926952,2.997290,1003.926952,1,True,7
4,LightGBMXT_BAG_L2,0.895085,roc_auc,110.244089,1823.134831,9.972903,142.961816,2,True,13
5,RandomForestEntr_BAG_L2,0.894231,roc_auc,110.091136,1870.259607,9.819950,190.086591,2,True,16
6,LightGBM_BAG_L2,0.892767,roc_auc,105.775692,1766.487950,5.504506,86.314935,2,True,14
7,RandomForestGini_BAG_L2,0.892413,roc_auc,110.705577,1852.869575,10.434392,172.696560,2,True,15
8,LightGBMXT_BAG_L1,0.892110,roc_auc,39.581874,235.790665,39.581874,235.790665,1,True,3
9,LightGBM_BAG_L1,0.889451,roc_auc,9.149957,83.271108,9.149957,83.271108,1,True,4


# Make a 'pure' submission file

In [5]:
prediction = automl.predict_proba(test)


data_submit = pd.read_csv('/kaggle/input/playground-series-s4e1/sample_submission.csv')
data_submit.Exited = prediction[1]
data_submit[['id', 'Exited']].to_csv('ag_processing.csv', index=False)


!head ag_processing.csv

INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


id,Exited
165034,0.014006898738443851
165035,0.8474394679069519
165036,0.012188786640763283
165037,0.21654006838798523
165038,0.3771299123764038
165039,0.0474398210644722
165040,0.01613999903202057
165041,0.06275671720504761
165042,0.6438615322113037


In [6]:
df_ext1 = pd.read_csv('/kaggle/input/playgrounds4e01-baseline-v2/Submission_V2.csv') 

# 0.89651


data_submit.Exited = 0.15 * data_submit.Exited + 0.85 * df_ext1.Exited
data_submit[['id', 'Exited']].to_csv('ensemble.csv', index=False)



!head ensemble.csv

id,Exited
165034,0.013495645096307192
165035,0.8311415965530612
165036,0.014482119205234933
165037,0.19539194467648313
165038,0.41019538211693723
165039,0.05249883677021676
165040,0.020357830593185786
165041,0.08376073464967672
165042,0.636160039999441
